In [ ]:
# Debug: Check the structure of downloaded data
test_data = yf.download(["VTI", "QQQM"], start="2018-01-01", progress=False)
print("Columns:", test_data.columns)
print("Column names level 0:", test_data.columns.get_level_values(0).unique() if isinstance(test_data.columns, pd.MultiIndex) else test_data.columns)
print("Data shape:", test_data.shape)
print(test_data.head())

In [2]:
%pip install yfinance

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     ------------- -------------------------- 1.0/3.0 MB 7.7 MB/s eta 0:00:01
     ---------------------------------------- 3.0/3.0 MB 10.1 MB/s  0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 11.9 MB/s  0:00:00
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15703 sha256=2c4d6b8e6572c91d158b8f0f18fbcafa387f25b74c4e4c26de0bb94821351588
  Stored in directory: c:\users\wongb

  DEPRECATION: Building 'multitasking' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'multitasking'. Discussion can be found at https://github.com/pypa/pip/issues/6334

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [118]:
my_tickers = [
    # Market & beta
    "VTI", "VT", "VXUS", "IEMG",

    # Adaptivity & defense
    "SPMO", "USMV", "DYNF",

    # Growth
    "VONG", "QQQ", "QQQM",

    # Value & income benchmarks
    "VTV", "VYM", "SCHD", "IWD",

    # Small value
    "AVUV", "AVDV",

    # Sector baselines
    "VDC",

    # Intl momentum
    "IMTM",

    # Equal weight control
    "RSP",

    # === SOLID GROWER / QUALITY / PROFITABILITY BATTLEFIELD ===
    "SPGP", "DUHP", "COWZ", "FNDX", "VFLO", "CALF",
    "QGRO", "QUAL", "QVAL", "VOOV", "MOAT", "GSLC", "QVAL", "DSTL", "DIVO",

    # Mid caps (conditional challengers)
    "VO", "VOT", "VOE", "IMCG", "IJR", "IJH", "MDY",

    # EM factors
    "AVEM", "EMGF", "EMXC", "EEMV"
]



In [119]:
import yfinance as yf
import pandas as pd
import numpy as np

def analyze_recovery_time(
    tickers,
    start_date="1990-01-01",
    end_date=None,
    sort_by="Annualized Drawdown Area (%-days/yr)",
    ascending=False,
    add_rank=True,
    use_adjusted=True,
):
    # 1. Fetch price data (auto-adjusted so we only work with adjusted values)
    raw_data = yf.download(
        tickers,
        start=start_date,
        end=end_date,
        progress=False,
        auto_adjust=True,
    )

    # When auto_adjust=True, yfinance always uses "Close" for successful tickers
    # (failed tickers appear in "Adj Close" but are all NaN)
    price_field = "Close"

    if len(tickers) > 1:
        data = raw_data[price_field]
    else:
        data = pd.DataFrame(raw_data[price_field])
    
    results = {}

    for ticker in tickers:
        if ticker not in data.columns:
            continue
        series = data[ticker].dropna()
        
        if len(series) == 0:
            continue
        
        # 2. Running Maximum and Drawdown
        running_max = series.cummax()
        drawdown = (series / running_max) - 1
        is_underwater = drawdown < 0

        # 3. Underwater duration stats (keep avg and p95 only)
        underwater_groups = (is_underwater != is_underwater.shift()).cumsum()
        underwater_only = underwater_groups[is_underwater]
        if not underwater_only.empty:
            durations = underwater_only.value_counts()
            avg_recovery_days = durations.mean()
            p95_recovery_days = float(durations.quantile(0.95))
        else:
            avg_recovery_days, p95_recovery_days = 0, 0
        
        total_days = len(series)

        # 4. Areas: drawdown area and ascent area; then ratio
        drawdown_area = float((-drawdown[drawdown < 0]).sum() * 100)
        annualized_drawdown_area = round(drawdown_area / (total_days / 252), 2) if total_days else 0

        daily_returns = series.pct_change()
        is_above_max = drawdown >= 0
        ascent_returns = daily_returns[is_above_max & (daily_returns > 0)]
        ascent_area = float(ascent_returns.sum() * 100) if len(ascent_returns) > 0 else 0
        annualized_ascent_area = round(ascent_area / (total_days / 252), 2) if total_days else 0

        ascent_drawdown_ratio = (
            round(annualized_ascent_area / annualized_drawdown_area, 3)
            if annualized_drawdown_area > 0 else np.nan
        )

        # 5. Risk metrics retained
        dd_neg = np.minimum(drawdown, 0)
        ulcer_index = float(np.sqrt((dd_neg ** 2).mean()) * 100) if len(dd_neg) else 0

        # Returns for Sortino and ascent slope
        returns = np.log(series / series.shift(1)).dropna()
        downside_returns = returns[returns < 0]
        downside_volatility = downside_returns.std() * np.sqrt(252) if len(downside_returns) > 0 else 0

        years = total_days / 252 if total_days else 0
        cagr = (series.iloc[-1] / series.iloc[0]) ** (1 / years) - 1 if years > 0 else np.nan

        risk_free_rate = 0.048  # 30-year proxy
        sortino_ratio = (cagr - risk_free_rate) / downside_volatility if downside_volatility > 0 else np.nan

        # Average slope of ascent streaks
        is_up = returns > 0
        up_groups = (is_up != is_up.shift()).cumsum()
        if is_up.any():
            slope_per_streak = []
            for streak_id in up_groups[is_up].unique():
                streak_mask = (up_groups == streak_id) & is_up
                streak_returns = returns[streak_mask]
                if len(streak_returns) > 0:
                    slope = (streak_returns.mean() * 100)
                    slope_per_streak.append(slope)
            avg_ascent_slope = round(np.mean(slope_per_streak), 3) if slope_per_streak else 0
        else:
            avg_ascent_slope = 0

        martin_ratio = cagr / (ulcer_index / 100) if ulcer_index not in (0, np.nan) else np.nan

        # Derived metric using avg recovery days
        return_over_recovery_time = (
            round(cagr * 100 / avg_recovery_days, 3)
            if avg_recovery_days not in (0, np.nan) else np.nan
        )
            
        results[ticker] = {
            # Kept core performance and risk
            "CAGR (%)": round(cagr * 100, 2) if not pd.isna(cagr) else np.nan,
            "Avg Recovery (Trading Days)": round(avg_recovery_days, 1),
            "P95 Recovery (Trading Days)": round(p95_recovery_days, 1),
            "Ascent/Drawdown Area Ratio": ascent_drawdown_ratio,
            "Sortino Ratio": round(sortino_ratio, 3) if not pd.isna(sortino_ratio) else np.nan,
            "Return / Recovery Time": return_over_recovery_time,
            "Avg Ascent Slope (% per day)": avg_ascent_slope,
            "Ulcer Index (%)": round(ulcer_index, 2),
            "Martin Ratio": round(martin_ratio, 3) if not pd.isna(martin_ratio) else np.nan,
            "Current Status": "Underwater" if drawdown.iloc[-1] < 0 else "At High"
        }

    df = pd.DataFrame(results).T

    # 6. Sort and rank by composite criteria (unchanged; columns must exist)
    sort_cols = ["Martin Ratio", "CAGR (%)", "Return / Recovery Time", "Avg Recovery (Trading Days)"]
    sort_ascending = [False, False, True]  # Higher Martin/CAGR better; lower recovery days better

    valid_sorts = [(col, asc) for col, asc in zip(sort_cols, sort_ascending) if col in df.columns]
    
    if valid_sorts:
        cols_to_sort = [col for col, _ in valid_sorts]
        ascending_list = [asc for _, asc in valid_sorts]
        df = df.sort_values(by=cols_to_sort, ascending=ascending_list)
        if add_rank:
            df.insert(0, "Rank", df[cols_to_sort[0]].rank(ascending=ascending_list[0], method="dense").astype(int))

    return df

# Full modern time scale
recovery_stats = analyze_recovery_time(my_tickers, start_date="1990-01-01", end_date=None)
from IPython.display import display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
display(recovery_stats.T)


,VFLO,DIVO,SPMO,USMV,SCHD,MOAT,DSTL,DUHP,FNDX,SPGP,VONG,VOOV,GSLC,QUAL,DYNF,COWZ,VDC,QGRO,AVDV,QQQM,AVUV,MDY,RSP,IMTM,IJH,VT,VO,VTI,IMCG,AVEM,IJR,VYM,VTV,EMGF,VOE,QVAL,VOT,CALF,VXUS,IWD,EMXC,EEMV,IEMG,QQQ
Rank,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44
CAGR (%),22.3,12.75,18.08,11.89,12.38,14.2,14.65,13.54,12.5,14.38,17.23,12.09,14.09,13.49,16.39,12.75,9.27,16.2,14.31,16.35,14.23,11.03,11.08,8.65,9.6,8.53,10.09,9.6,11.06,10.05,9.69,9.01,9.27,8.75,9.06,8.16,10.06,8.69,6.1,7.78,6.78,4.67,5.21,10.5
Avg Recovery (Trading Days),12.1,13.4,15.1,12.5,14.8,13.8,12.8,14.9,13.9,13.5,12.3,14.5,12.0,13.1,12.5,19.4,16.8,17.4,17.8,17.7,31.8,21.7,16.9,26.0,24.8,20.3,19.4,18.0,20.9,29.9,27.5,17.6,17.3,36.8,21.3,33.0,20.0,40.9,34.3,20.8,44.6,59.0,51.4,31.6
P95 Recovery (Trading Days),62.6,69.0,59.7,42.9,57.8,57.8,46.6,56.8,53.4,54.5,55.6,44.5,48.4,52.8,39.0,109.2,61.6,63.8,44.9,39.0,199.1,84.6,63.2,75.3,101.9,40.0,64.5,55.0,61.4,119.6,102.8,52.6,45.0,94.8,59.1,65.0,65.7,286.7,85.8,64.0,157.8,353.9,343.9,57.4
Ascent/Drawdown Area Ratio,0.043,0.027,0.02,0.023,0.02,0.021,0.02,0.017,0.019,0.019,0.019,0.017,0.017,0.017,0.015,0.014,0.015,0.011,0.011,0.01,0.009,0.009,0.009,0.009,0.007,0.007,0.007,0.007,0.006,0.005,0.006,0.007,0.007,0.005,0.006,0.005,0.006,0.005,0.004,0.005,0.004,0.003,0.002,0.002
Sortino Ratio,1.552,0.62,0.743,0.616,0.604,0.677,0.596,0.716,0.543,0.577,0.792,0.536,0.619,0.614,0.683,0.485,0.406,0.612,0.577,0.712,0.424,0.365,0.38,0.229,0.283,0.212,0.307,0.307,0.356,0.324,0.283,0.274,0.283,0.251,0.237,0.192,0.282,0.201,0.09,0.19,0.119,-0.012,0.028,0.271
Return / Recovery Time,1.845,0.953,1.196,0.949,0.835,1.027,1.149,0.91,0.899,1.066,1.402,0.831,1.173,1.033,1.316,0.656,0.553,0.929,0.803,0.922,0.448,0.509,0.656,0.333,0.386,0.42,0.519,0.534,0.529,0.336,0.352,0.513,0.536,0.238,0.425,0.247,0.502,0.213,0.178,0.374,0.152,0.079,0.101,0.332
Avg Ascent Slope (% per day),0.772,0.655,0.958,0.59,0.684,0.794,0.858,0.806,0.731,0.844,0.85,0.706,0.777,0.758,0.895,0.862,0.635,1.07,0.843,1.061,1.281,0.949,0.839,0.824,0.944,0.852,0.868,0.82,0.917,0.928,1.051,0.766,0.791,0.94,0.887,0.967,0.962,1.207,0.786,0.808,0.898,0.648,0.9,1.178
Ulcer Index (%),3.8,4.25,7.16,4.83,5.16,5.93,6.38,6.26,5.85,6.93,8.48,6.13,7.37,7.18,9.19,7.23,6.11,11.58,10.49,13.26,11.87,11.18,12.0,9.66,11.86,11.15,13.32,12.74,14.79,13.79,13.63,12.72,13.75,13.39,14.16,12.79,15.89,13.99,10.81,14.11,13.27,10.19,16.0,43.9
Martin Ratio,5.862,3.003,2.525,2.463,2.399,2.395,2.296,2.162,2.137,2.076,2.032,1.972,1.912,1.879,1.784,1.763,1.516,1.399,1.364,1.234,1.199,0.987,0.923,0.895,0.81,0.765,0.758,0.754,0.748,0.729,0.711,0.708,0.674,0.654,0.64,0.638,0.633,0.622,0.565,0.552,0.511,0.458,0.325,0.239


In [137]:
recovery_stats = analyze_recovery_time(my_tickers, start_date="2020-01-01", end_date=None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
display(recovery_stats.T)

,VFLO,DIVO,SPMO,DUHP,DSTL,FNDX,COWZ,MOAT,DYNF,VYM,GSLC,QQQ,VTV,VTI,VONG,SCHD,VOOV,QGRO,QUAL,VDC,RSP,VT,SPGP,QQQM,IWD,USMV,AVDV,AVUV,IMTM,VOE,IJH,MDY,VO,QVAL,VXUS,IMCG,CALF,VOT,EMXC,IJR,AVEM,EMGF,EEMV,IEMG
Rank,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,36,37,38,39,40,41,42,43
CAGR (%),22.3,12.38,20.73,13.54,13.0,13.81,14.3,12.67,16.05,10.89,14.32,20.15,10.97,14.55,18.96,10.05,11.04,17.53,13.56,7.51,10.9,11.99,11.81,16.35,9.86,8.24,12.64,13.46,10.98,9.58,10.21,9.93,10.41,10.45,8.38,11.45,10.44,10.85,8.08,8.35,8.37,7.33,4.06,6.3
Avg Recovery (Trading Days),12.1,13.7,13.3,14.9,14.1,14.9,19.3,16.6,13.3,17.6,13.1,16.1,15.3,12.7,14.9,20.9,16.1,16.9,13.5,25.4,18.1,15.1,19.0,17.7,18.0,20.0,20.1,35.1,20.2,21.5,30.8,32.2,19.8,26.7,25.7,26.3,36.8,24.0,36.5,48.5,35.4,38.5,53.4,49.9
P95 Recovery (Trading Days),62.6,76.8,40.2,56.8,56.1,48.8,100.0,79.0,38.6,70.6,36.2,61.0,48.9,32.0,54.4,65.2,84.0,42.8,37.7,97.6,44.0,38.3,81.9,39.0,52.8,53.6,55.3,225.0,64.3,69.5,155.8,166.3,31.0,121.4,77.3,82.0,209.3,87.8,178.5,237.9,154.5,171.0,197.2,128.4
Ascent/Drawdown Area Ratio,0.043,0.025,0.018,0.017,0.017,0.016,0.014,0.014,0.013,0.014,0.013,0.012,0.014,0.013,0.012,0.013,0.013,0.01,0.012,0.012,0.011,0.011,0.011,0.01,0.011,0.009,0.009,0.009,0.009,0.009,0.008,0.008,0.007,0.007,0.007,0.005,0.007,0.005,0.005,0.004,0.004,0.004,0.003,0.002
Sortino Ratio,1.552,0.556,0.84,0.716,0.478,0.521,0.518,0.461,0.642,0.386,0.554,0.772,0.377,0.547,0.736,0.338,0.375,0.673,0.519,0.207,0.341,0.426,0.345,0.712,0.294,0.235,0.47,0.385,0.391,0.257,0.268,0.256,0.303,0.269,0.223,0.359,0.265,0.312,0.182,0.177,0.218,0.156,-0.059,0.087
Return / Recovery Time,1.845,0.904,1.557,0.91,0.922,0.929,0.742,0.763,1.204,0.618,1.095,1.25,0.719,1.147,1.271,0.48,0.684,1.04,1.005,0.296,0.6,0.796,0.62,0.922,0.547,0.412,0.628,0.384,0.542,0.446,0.332,0.309,0.527,0.392,0.326,0.435,0.284,0.453,0.221,0.172,0.236,0.19,0.076,0.126
Avg Ascent Slope (% per day),0.772,0.707,1.056,0.806,0.892,0.884,0.959,0.992,0.939,0.841,0.925,1.179,0.839,0.917,1.12,0.841,0.834,1.126,0.955,0.725,0.901,0.856,1.019,1.061,0.882,0.723,0.851,1.309,0.879,0.914,1.076,1.087,0.961,1.122,0.84,1.085,1.327,1.111,0.915,1.208,0.944,0.949,0.609,0.972
Ulcer Index (%),3.8,4.64,8.58,6.26,6.76,7.22,7.95,7.62,9.75,6.69,9.09,12.85,7.11,9.49,12.47,6.71,7.48,12.39,9.64,5.43,8.45,9.3,9.17,13.26,8.05,6.87,10.72,12.0,10.57,9.51,10.33,10.4,11.45,11.73,10.61,15.05,13.71,16.49,12.72,13.63,14.11,12.57,9.33,17.6
Martin Ratio,5.862,2.672,2.416,2.162,1.922,1.914,1.799,1.662,1.646,1.627,1.576,1.569,1.543,1.534,1.521,1.496,1.477,1.415,1.407,1.382,1.29,1.289,1.288,1.234,1.224,1.2,1.179,1.122,1.039,1.008,0.988,0.956,0.909,0.891,0.79,0.761,0.761,0.658,0.635,0.613,0.593,0.583,0.435,0.358


In [120]:
# Named historical periods with (start_date, end_date)
periods = {
    "DotCom Bubble Crash": ("2000-02-10", "2005-01-15"),
    "2005-2007 International Bull Run": ("2005-01-16", "2007-09-08"),
    "GFC": ("2007-09-09", "2013-02-25"),
    "2013-2015 Recovery Bull Run": ("2013-02-26", "2015-05-22"),
    "2015-2016 Stock Market Selloff": ("2015-05-23", "2016-08-22"),
    "2016-2018 Bull Run": ("2016-08-23", "2018-07-19"),
    "2018 Q4 Correction": ("2018-07-20", "2019-08-19"),
    "2019 Rebound": ("2019-08-20", "2020-01-18"),
    "COVID-19 Pandemic": ("2020-01-19", "2021-09-21"),
    "2022 Rate Hike Drawdown": ("2021-10-05", "2024-02-26"),
    "2024–2025 Bull Run": ("2024-02-27", "2025-01-18"),
    "2025 Trump Trade War": ("2025-01-19", "2025-07-26"),
    "AI Boom": ("2025-07-27", None),
}

In [121]:
from collections import OrderedDict
from IPython.display import display, Markdown

def summarize_periods(periods: dict, tickers: list, top_n: int = 10):
    """
    Run analyze_recovery_time for each named period and return the top-N ranked rows.

    Parameters:
    - periods: {period_name: (start_date, end_date)} with end_date possibly None
    - tickers: list of tickers to analyze
    - top_n: number of top-ranked rows to keep (default 7)

    Returns:
    - OrderedDict { period_name: DataFrame_of_top_n }
    """
    summaries = OrderedDict()
    for name, (start_date, end_date) in periods.items():
        df = analyze_recovery_time(tickers, start_date=start_date, end_date=end_date)
        if isinstance(df, pd.DataFrame) and not df.empty:
            if "Rank" in df.columns:
                df = df.sort_values(by="Rank", ascending=True)
            top_df = df.head(top_n)
            summaries[name] = top_df
        else:
            summaries[name] = pd.DataFrame()
    return summaries


def display_period_summaries(summaries: dict, transpose: bool = True):
    """Display each period’s summary table (optionally transposed)."""
    for name, df in summaries.items():
        display(Markdown(f"### {name} — Top {min(len(df), 7)}"))
        if df is None or df.empty:
            display(Markdown("_No data available for this period._"))
        else:
            display(df.T if transpose else df)

In [122]:
# Example: summarize and display the top-7 for all named periods
# Run this cell to generate the summaries.
summaries = summarize_periods(periods, my_tickers, top_n=7)
display_period_summaries(summaries, transpose=True)

$DIVO: possibly delisted; no price data found  (1d 2000-02-10 -> 2005-01-15) (Yahoo error = "Data doesn't exist for startDate = 950158800, endDate = 1105765200")
$AVUV: possibly delisted; no price data found  (1d 2000-02-10 -> 2005-01-15) (Yahoo error = "Data doesn't exist for startDate = 950158800, endDate = 1105765200")
$CALF: possibly delisted; no price data found  (1d 2000-02-10 -> 2005-01-15) (Yahoo error = "Data doesn't exist for startDate = 950158800, endDate = 1105765200")
$EEMV: possibly delisted; no price data found  (1d 2000-02-10 -> 2005-01-15) (Yahoo error = "Data doesn't exist for startDate = 950158800, endDate = 1105765200")
$IMTM: possibly delisted; no price data found  (1d 2000-02-10 -> 2005-01-15) (Yahoo error = "Data doesn't exist for startDate = 950158800, endDate = 1105765200")
$QQQM: possibly delisted; no price data found  (1d 2000-02-10 -> 2005-01-15) (Yahoo error = "Data doesn't exist for startDate = 950158800, endDate = 1105765200")
$MOAT: possibly delisted; no

### DotCom Bubble Crash — Top 7

,RSP,IMCG,VTV,VO,VDC,IJR,IJH
Rank,1,2,3,4,5,6,7
CAGR (%),27.2,27.4,11.75,14.29,10.59,12.06,8.59
Avg Recovery (Trading Days),8.6,5.9,12.6,14.6,16.3,23.8,32.9
P95 Recovery (Trading Days),26.1,20.5,40.4,61.1,71.2,87.2,238.2
Ascent/Drawdown Area Ratio,0.077,0.087,0.034,0.032,0.024,0.009,0.006
Sortino Ratio,2.683,2.42,1.0,1.177,0.998,0.563,0.29
Return / Recovery Time,3.172,4.615,0.929,0.981,0.65,0.507,0.261
Avg Ascent Slope (% per day),0.666,0.671,0.469,0.612,0.49,1.045,1.001
Ulcer Index (%),2.75,3.05,2.76,3.94,3.44,12.35,11.89
Martin Ratio,9.874,8.977,4.25,3.627,3.078,0.976,0.722


### 2005-2007 International Bull Run — Top 7

,VOT,VDC,VTV,IWD,VTI,VO,RSP
Rank,1,2,3,4,5,6,7
CAGR (%),20.99,8.53,12.08,11.77,10.54,13.59,10.43
Avg Recovery (Trading Days),7.1,10.7,9.1,8.7,12.1,12.4,11.0
P95 Recovery (Trading Days),29.3,44.8,39.6,44.6,63.0,38.2,42.6
Ascent/Drawdown Area Ratio,0.085,0.048,0.05,0.049,0.038,0.035,0.035
Sortino Ratio,1.415,0.575,0.807,0.739,0.658,0.906,0.613
Return / Recovery Time,2.937,0.794,1.322,1.354,0.872,1.1,0.945
Avg Ascent Slope (% per day),0.662,0.465,0.551,0.548,0.54,0.62,0.582
Ulcer Index (%),3.0,1.73,2.63,2.69,2.89,4.0,3.31
Martin Ratio,7.0,4.925,4.587,4.373,3.642,3.399,3.153


...

,DYNF,SPMO,VTI,QGRO,GSLC,VYM,DIVO
Rank,1,2,3,4,5,6,7
CAGR (%),25.76,33.32,21.68,29.04,20.97,19.83,16.5
Avg Recovery (Trading Days),8.3,11.6,7.2,10.4,7.4,7.5,8.4
P95 Recovery (Trading Days),37.0,42.0,31.0,31.8,31.5,32.0,28.0
Ascent/Drawdown Area Ratio,0.093,0.069,0.084,0.069,0.087,0.078,0.078
Sortino Ratio,1.94,2.14,1.667,2.118,1.637,1.966,1.82
Return / Recovery Time,3.091,2.866,3.028,2.796,2.843,2.644,1.957
Avg Ascent Slope (% per day),0.668,0.946,0.634,0.79,0.623,0.537,0.433
Ulcer Index (%),2.35,3.44,2.26,3.06,2.29,2.18,2.05
Martin Ratio,10.963,9.692,9.59,9.494,9.171,9.077,8.039


### 2025 Trump Trade War — Top 7

,AVDV,IMTM,VXUS,AVEM,EMGF,IEMG,EEMV
Rank,1,2,3,4,5,6,7
CAGR (%),59.57,41.37,37.37,42.58,38.05,37.34,19.56
Avg Recovery (Trading Days),4.6,6.1,5.2,4.6,4.5,6.1,6.7
P95 Recovery (Trading Days),11.6,17.2,14.0,17.5,16.9,21.1,22.5
Ascent/Drawdown Area Ratio,0.218,0.155,0.148,0.121,0.127,0.106,0.114
Sortino Ratio,2.963,2.163,2.065,2.245,2.096,2.043,1.394
Return / Recovery Time,13.01,6.754,7.22,9.298,8.455,6.155,2.934
Avg Ascent Slope (% per day),0.794,0.807,0.827,0.948,0.907,0.931,0.506
Ulcer Index (%),2.68,2.43,2.71,3.24,3.09,3.26,1.76
Martin Ratio,22.222,17.011,13.815,13.128,12.324,11.463,11.086


### AI Boom — Top 7

,AVDV,EMXC,DIVO,FNDX,VXUS,DYNF,VTV
Rank,1,2,3,4,5,6,7
CAGR (%),49.18,42.13,23.13,26.87,28.77,26.68,21.16
Avg Recovery (Trading Days),4.7,7.1,4.4,5.3,5.0,7.3,6.0
P95 Recovery (Trading Days),11.6,25.5,10.0,13.3,12.4,19.5,12.0
Ascent/Drawdown Area Ratio,0.307,0.161,0.244,0.202,0.192,0.157,0.173
Sortino Ratio,5.952,3.97,3.789,3.221,3.301,2.441,2.797
Return / Recovery Time,10.538,5.942,5.242,5.038,5.753,3.668,3.527
Avg Ascent Slope (% per day),0.501,0.729,0.472,0.538,0.548,0.603,0.601
Ulcer Index (%),1.14,1.67,0.94,1.1,1.29,1.27,1.05
Martin Ratio,43.192,25.233,24.716,24.423,22.305,21.076,20.218


In [123]:
# Print each period with comma-separated top-7 tickers (descending by Rank)
for period_name, df in summaries.items():
    if df is None or df.empty:
        print(f"{period_name}:")
        continue
    df_desc = df.sort_values(by="Rank", ascending=True).head(10) if "Rank" in df.columns else df.head(7)
    tickers_asc = df_desc.index.tolist()
    print(f"{period_name}: " + ", ".join(tickers_asc))

DotCom Bubble Crash: RSP, IMCG, VTV, VO, VDC, IJR, IJH
2005-2007 International Bull Run: VOT, VDC, VTV, IWD, VTI, VO, RSP
GFC: IEMG, USMV, SCHD, EEMV, MOAT, VONG, SPGP
2013-2015 Recovery Bull Run: QQQ, QVAL, VONG, SPGP, IMTM, VTI, VOE
2015-2016 Stock Market Selloff: EMGF, USMV, VDC, GSLC, SPMO, SCHD, VYM
2016-2018 Bull Run: SPGP, QQQ, SPMO, VONG, IMCG, IJR, VOT
2018 Q4 Correction: USMV, DYNF, VDC, DSTL, DIVO, MOAT, SCHD
2019 Rebound: QUAL, QQQ, AVDV, GSLC, MOAT, VOOV, VT
COVID-19 Pandemic: QQQM, QQQ, QGRO, VONG, IMCG, SPMO, VOT
2022 Rate Hike Drawdown: VFLO, DIVO, VOOV, COWZ, DUHP, DSTL, FNDX
2024–2025 Bull Run: DYNF, SPMO, VTI, QGRO, GSLC, VYM, DIVO
2025 Trump Trade War: AVDV, IMTM, VXUS, AVEM, EMGF, IEMG, EEMV
AI Boom: AVDV, EMXC, DIVO, FNDX, VXUS, DYNF, VTV


In [124]:
# DotCom Bubble Crash
recovery_stats = analyze_recovery_time(my_tickers, start_date="2000-02-10", end_date="2005-01-15")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
display(recovery_stats.T)

$AVUV: possibly delisted; no price data found  (1d 2000-02-10 -> 2005-01-15) (Yahoo error = "Data doesn't exist for startDate = 950158800, endDate = 1105765200")
$DIVO: possibly delisted; no price data found  (1d 2000-02-10 -> 2005-01-15) (Yahoo error = "Data doesn't exist for startDate = 950158800, endDate = 1105765200")
$MOAT: possibly delisted; no price data found  (1d 2000-02-10 -> 2005-01-15) (Yahoo error = "Data doesn't exist for startDate = 950158800, endDate = 1105765200")
$SPGP: possibly delisted; no price data found  (1d 2000-02-10 -> 2005-01-15) (Yahoo error = "Data doesn't exist for startDate = 950158800, endDate = 1105765200")
$SCHD: possibly delisted; no price data found  (1d 2000-02-10 -> 2005-01-15) (Yahoo error = "Data doesn't exist for startDate = 950158800, endDate = 1105765200")
$EEMV: possibly delisted; no price data found  (1d 2000-02-10 -> 2005-01-15) (Yahoo error = "Data doesn't exist for startDate = 950158800, endDate = 1105765200")
$QGRO: possibly delisted; no

,RSP,IMCG,VTV,VO,VDC,IJR,IJH,MDY,IWD,VTI,QQQ
Rank,1,2,3,4,5,6,7,8,9,10,11
CAGR (%),27.2,27.4,11.75,14.29,10.59,12.06,8.59,8.33,5.48,2.47,-17.88
Avg Recovery (Trading Days),8.6,5.9,12.6,14.6,16.3,23.8,32.9,28.9,33.3,54.3,204.7
P95 Recovery (Trading Days),26.1,20.5,40.4,61.1,71.2,87.2,238.2,158.4,89.4,284.8,907.5
Ascent/Drawdown Area Ratio,0.077,0.087,0.034,0.032,0.024,0.009,0.006,0.006,0.005,0.002,0.0
Sortino Ratio,2.683,2.42,1.0,1.177,0.998,0.563,0.29,0.249,0.058,-0.189,-0.808
Return / Recovery Time,3.172,4.615,0.929,0.981,0.65,0.507,0.261,0.288,0.165,0.045,-0.087
Avg Ascent Slope (% per day),0.666,0.671,0.469,0.612,0.49,1.045,1.001,1.083,0.795,0.857,1.854
Ulcer Index (%),2.75,3.05,2.76,3.94,3.44,12.35,11.89,11.63,11.97,14.69,64.95
Martin Ratio,9.874,8.977,4.25,3.627,3.078,0.976,0.722,0.716,0.458,0.168,-0.275


In [125]:
# 2005-2007 International Bull Run
recovery_stats = analyze_recovery_time(my_tickers, start_date="2005-01-16", end_date="2007-09-08")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
display(recovery_stats.T)

$EEMV: possibly delisted; no price data found  (1d 2005-01-16 -> 2007-09-08) (Yahoo error = "Data doesn't exist for startDate = 1105851600, endDate = 1189224000")
$QQQM: possibly delisted; no price data found  (1d 2005-01-16 -> 2007-09-08) (Yahoo error = "Data doesn't exist for startDate = 1105851600, endDate = 1189224000")
$MOAT: possibly delisted; no price data found  (1d 2005-01-16 -> 2007-09-08) (Yahoo error = "Data doesn't exist for startDate = 1105851600, endDate = 1189224000")
$SCHD: possibly delisted; no price data found  (1d 2005-01-16 -> 2007-09-08) (Yahoo error = "Data doesn't exist for startDate = 1105851600, endDate = 1189224000")
$DIVO: possibly delisted; no price data found  (1d 2005-01-16 -> 2007-09-08) (Yahoo error = "Data doesn't exist for startDate = 1105851600, endDate = 1189224000")
$QGRO: possibly delisted; no price data found  (1d 2005-01-16 -> 2007-09-08) (Yahoo error = "Data doesn't exist for startDate = 1105851600, endDate = 1189224000")
$SPGP: possibly delist

,VOT,VDC,VTV,IWD,VTI,VO,RSP,IMCG,MDY,VOE,IJH,VYM,IJR,QQQ
Rank,1,2,3,4,5,6,7,8,9,10,11,12,13,14
CAGR (%),20.99,8.53,12.08,11.77,10.54,13.59,10.43,15.88,12.32,11.52,12.3,6.87,11.13,9.01
Avg Recovery (Trading Days),7.1,10.7,9.1,8.7,12.1,12.4,11.0,11.7,15.3,7.5,14.9,8.0,13.9,28.4
P95 Recovery (Trading Days),29.3,44.8,39.6,44.6,63.0,38.2,42.6,33.5,68.6,32.5,68.2,36.2,68.3,122.0
Ascent/Drawdown Area Ratio,0.085,0.048,0.05,0.049,0.038,0.035,0.035,0.028,0.028,0.054,0.027,0.057,0.024,0.013
Sortino Ratio,1.415,0.575,0.807,0.739,0.658,0.906,0.613,1.123,0.781,0.58,0.748,0.195,0.574,0.432
Return / Recovery Time,2.937,0.794,1.322,1.354,0.872,1.1,0.945,1.359,0.804,1.544,0.826,0.859,0.799,0.317
Avg Ascent Slope (% per day),0.662,0.465,0.551,0.548,0.54,0.62,0.582,0.684,0.645,0.555,0.674,0.515,0.765,0.715
Ulcer Index (%),3.0,1.73,2.63,2.69,2.89,4.0,3.31,5.24,4.34,4.07,4.44,2.6,4.92,5.56
Martin Ratio,7.0,4.925,4.587,4.373,3.642,3.399,3.153,3.029,2.836,2.829,2.772,2.643,2.261,1.62


In [126]:
# GFC
recovery_stats = analyze_recovery_time(my_tickers, start_date="2007-09-09", end_date="2013-02-25")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
display(recovery_stats.T)

$CALF: possibly delisted; no price data found  (1d 2007-09-09 -> 2013-02-25) (Yahoo error = "Data doesn't exist for startDate = 1189310400, endDate = 1361768400")
$QQQM: possibly delisted; no price data found  (1d 2007-09-09 -> 2013-02-25) (Yahoo error = "Data doesn't exist for startDate = 1189310400, endDate = 1361768400")
$EMGF: possibly delisted; no price data found  (1d 2007-09-09 -> 2013-02-25) (Yahoo error = "Data doesn't exist for startDate = 1189310400, endDate = 1361768400")
$AVUV: possibly delisted; no price data found  (1d 2007-09-09 -> 2013-02-25) (Yahoo error = "Data doesn't exist for startDate = 1189310400, endDate = 1361768400")
$IMTM: possibly delisted; no price data found  (1d 2007-09-09 -> 2013-02-25) (Yahoo error = "Data doesn't exist for startDate = 1189310400, endDate = 1361768400")
$DIVO: possibly delisted; no price data found  (1d 2007-09-09 -> 2013-02-25) (Yahoo error = "Data doesn't exist for startDate = 1189310400, endDate = 1361768400")
$EMXC: possibly delist

,IEMG,USMV,SCHD,EEMV,MOAT,VONG,SPGP,VOOV,VDC,QQQ,IJH,MDY,IJR,RSP,VOE,VO,VTI,IMCG,VOT,VYM,VT,VXUS,IWD,VTV
Rank,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
CAGR (%),23.1,18.63,19.04,21.19,16.45,15.22,13.62,15.49,9.69,7.08,6.49,6.3,5.57,5.04,4.95,4.57,3.67,3.93,3.94,3.58,2.7,1.47,1.7,1.59
Avg Recovery (Trading Days),10.7,6.8,7.8,13.6,14.8,12.6,18.4,14.4,19.2,33.8,36.1,36.1,47.3,41.0,43.9,45.2,55.5,66.2,52.9,44.0,56.9,55.9,123.1,150.4
P95 Recovery (Trading Days),31.0,26.7,34.4,53.0,67.9,76.0,89.2,53.6,62.9,107.2,140.2,142.2,133.8,139.1,170.7,149.8,91.5,397.6,254.4,58.9,430.6,282.0,669.5,805.0
Ascent/Drawdown Area Ratio,0.104,0.109,0.075,0.047,0.031,0.023,0.018,0.019,0.009,0.003,0.003,0.003,0.002,0.002,0.002,0.002,0.001,0.001,0.001,0.001,0.001,0.002,0.0,0.0
Sortino Ratio,2.742,2.103,1.699,1.331,1.354,0.742,0.576,0.783,0.374,0.117,0.076,0.063,0.034,0.011,0.007,-0.01,-0.055,-0.037,-0.036,-0.063,-0.088,-0.19,-0.144,-0.149
Return / Recovery Time,2.166,2.744,2.443,1.561,1.109,1.207,0.74,1.079,0.505,0.21,0.18,0.174,0.118,0.123,0.113,0.101,0.066,0.059,0.075,0.081,0.047,0.026,0.014,0.011
Avg Ascent Slope (% per day),0.617,0.537,0.63,0.854,0.833,0.813,0.954,0.789,0.746,1.221,1.268,1.278,1.397,1.27,1.28,1.246,1.139,1.174,1.228,1.007,1.22,1.032,1.166,1.191
Ulcer Index (%),1.79,1.71,2.07,3.54,4.71,5.29,5.68,6.78,9.62,19.91,19.26,19.45,21.05,21.3,21.68,22.4,21.59,23.56,23.73,22.37,17.64,14.25,26.33,25.99
Martin Ratio,12.932,10.924,9.21,5.988,3.495,2.875,2.398,2.285,1.008,0.356,0.337,0.324,0.265,0.237,0.228,0.204,0.17,0.167,0.166,0.16,0.153,0.103,0.064,0.061


In [127]:
# 2013-2015 Recovery Bull Run
recovery_stats = analyze_recovery_time(my_tickers, start_date="2013-02-26", end_date="2015-05-22")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
display(recovery_stats.T)

$QQQM: possibly delisted; no price data found  (1d 2013-02-26 -> 2015-05-22) (Yahoo error = "Data doesn't exist for startDate = 1361854800, endDate = 1432267200")
$EMXC: possibly delisted; no price data found  (1d 2013-02-26 -> 2015-05-22) (Yahoo error = "Data doesn't exist for startDate = 1361854800, endDate = 1432267200")
$CALF: possibly delisted; no price data found  (1d 2013-02-26 -> 2015-05-22) (Yahoo error = "Data doesn't exist for startDate = 1361854800, endDate = 1432267200")
$EMGF: possibly delisted; no price data found  (1d 2013-02-26 -> 2015-05-22) (Yahoo error = "Data doesn't exist for startDate = 1361854800, endDate = 1432267200")
$DIVO: possibly delisted; no price data found  (1d 2013-02-26 -> 2015-05-22) (Yahoo error = "Data doesn't exist for startDate = 1361854800, endDate = 1432267200")
$AVUV: possibly delisted; no price data found  (1d 2013-02-26 -> 2015-05-22) (Yahoo error = "Data doesn't exist for startDate = 1361854800, endDate = 1432267200")
$QGRO: possibly delist

,QQQ,QVAL,VONG,SPGP,IMTM,VTI,VOE,RSP,USMV,QUAL,VO,VTV,VYM,IWD,SCHD,VOOV,VOT,FNDX,VDC,IMCG,IJH,MDY,MOAT,IJR,VT,VXUS,EEMV,IEMG
Rank,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28
CAGR (%),27.1,23.79,21.16,20.8,29.11,19.51,21.06,20.44,16.86,17.09,21.22,18.33,17.27,17.56,16.85,17.28,20.72,15.39,16.48,20.26,18.48,18.27,17.72,18.87,13.91,9.09,3.24,2.38
Avg Recovery (Trading Days),7.6,6.8,7.9,7.6,12.4,8.1,8.9,8.1,8.0,8.9,8.2,7.8,7.2,8.3,9.3,8.8,8.4,8.2,10.5,9.3,10.0,10.4,11.3,13.2,13.0,24.6,44.6,35.9
P95 Recovery (Trading Days),36.2,26.0,32.0,31.7,16.4,30.9,35.5,31.7,31.9,28.0,34.7,32.0,31.1,32.4,34.6,34.0,32.7,32.9,51.0,35.8,33.5,33.6,35.0,39.8,38.0,89.0,222.2,205.9
Ascent/Drawdown Area Ratio,0.099,0.109,0.089,0.095,0.382,0.082,0.081,0.081,0.087,0.077,0.075,0.08,0.078,0.078,0.069,0.071,0.065,0.071,0.066,0.056,0.064,0.061,0.055,0.045,0.041,0.013,0.004,0.004
Sortino Ratio,2.269,2.429,1.97,2.048,0.428,1.763,1.918,1.831,1.754,1.575,1.761,1.727,1.66,1.589,1.639,1.662,1.64,1.414,1.599,1.618,1.456,1.418,1.54,1.422,1.077,0.47,-0.178,-0.224
Return / Recovery Time,3.546,3.487,2.682,2.755,2.347,2.422,2.377,2.508,2.103,1.92,2.576,2.344,2.388,2.11,1.813,1.958,2.481,1.868,1.575,2.182,1.84,1.761,1.563,1.427,1.072,0.369,0.073,0.066
Avg Ascent Slope (% per day),0.637,0.791,0.579,0.576,1.481,0.573,0.61,0.603,0.511,0.556,0.638,0.583,0.544,0.567,0.529,0.546,0.681,0.555,0.539,0.679,0.629,0.649,0.589,0.766,0.568,0.639,0.627,0.82
Ulcer Index (%),2.13,2.06,1.86,1.83,2.7,1.84,2.01,1.95,1.67,1.72,2.16,1.89,1.85,1.93,1.89,1.96,2.44,1.85,2.03,2.68,2.46,2.46,2.53,3.12,2.59,5.06,7.04,7.7
Martin Ratio,12.736,11.576,11.399,11.374,10.776,10.625,10.482,10.471,10.122,9.931,9.805,9.696,9.351,9.096,8.906,8.806,8.504,8.297,8.13,7.557,7.516,7.421,7.002,6.049,5.374,1.797,0.46,0.309


In [128]:
# 2015-2016 Stock Market Selloff
recovery_stats = analyze_recovery_time(my_tickers, start_date="2015-05-23", end_date="2016-08-22")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
display(recovery_stats.T)

$DIVO: possibly delisted; no price data found  (1d 2015-05-23 -> 2016-08-22) (Yahoo error = "Data doesn't exist for startDate = 1432353600, endDate = 1471838400")
$EMXC: possibly delisted; no price data found  (1d 2015-05-23 -> 2016-08-22) (Yahoo error = "Data doesn't exist for startDate = 1432353600, endDate = 1471838400")
$QQQM: possibly delisted; no price data found  (1d 2015-05-23 -> 2016-08-22) (Yahoo error = "Data doesn't exist for startDate = 1432353600, endDate = 1471838400")
$AVUV: possibly delisted; no price data found  (1d 2015-05-23 -> 2016-08-22) (Yahoo error = "Data doesn't exist for startDate = 1432353600, endDate = 1471838400")
$QGRO: possibly delisted; no price data found  (1d 2015-05-23 -> 2016-08-22) (Yahoo error = "Data doesn't exist for startDate = 1432353600, endDate = 1471838400")
$CALF: possibly delisted; no price data found  (1d 2015-05-23 -> 2016-08-22) (Yahoo error = "Data doesn't exist for startDate = 1432353600, endDate = 1471838400")
$DYNF: possibly delist

,EMGF,USMV,VDC,GSLC,SPMO,SCHD,VYM,MOAT,QUAL,QQQ,VONG,VTV,VTI,FNDX,IJR,RSP,VOOV,IWD,VOE,IJH,MDY,SPGP,VO,VOT,VT,IMCG,IMTM,IEMG,VXUS,EEMV,QVAL
Rank,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
CAGR (%),22.74,11.65,11.97,11.57,10.26,8.99,7.42,10.54,6.12,7.03,4.99,5.1,4.19,4.11,5.89,4.1,3.91,3.75,3.5,3.62,3.39,1.88,1.87,-0.01,-0.43,-1.75,-2.6,-6.23,-5.27,-6.6,-9.34
Avg Recovery (Trading Days),20.1,12.2,10.2,10.0,90.5,16.4,18.9,15.6,20.6,29.4,22.5,19.3,21.1,26.9,24.5,29.8,24.6,22.6,27.3,24.8,24.8,50.7,43.7,103.3,312.0,103.3,313.0,312.0,312.0,313.0,154.0
P95 Recovery (Trading Days),56.7,41.4,37.7,32.2,154.8,91.6,78.3,43.0,83.6,121.1,107.8,78.6,97.3,128.5,122.1,151.3,109.7,105.0,141.0,123.8,123.8,201.0,195.6,266.1,312.0,266.1,313.0,312.0,312.0,313.0,291.7
Ascent/Drawdown Area Ratio,0.033,0.036,0.042,0.031,0.022,0.021,0.018,0.016,0.014,0.008,0.009,0.01,0.007,0.008,0.006,0.005,0.006,0.007,0.004,0.004,0.004,0.004,0.003,0.001,0.0,0.001,0.0,0.0,0.0,0.0,0.0
Sortino Ratio,0.63,0.73,0.769,0.659,0.345,0.41,0.238,0.463,0.111,0.152,0.015,0.026,-0.05,-0.06,0.09,-0.057,-0.077,-0.088,-0.102,-0.098,-0.115,-0.21,-0.228,-0.357,-0.395,-0.482,-0.356,-0.716,-0.656,-0.934,-1.056
Return / Recovery Time,1.129,0.953,1.176,1.163,0.113,0.55,0.392,0.678,0.296,0.239,0.221,0.264,0.198,0.153,0.24,0.137,0.159,0.166,0.128,0.146,0.136,0.037,0.043,-0.0,-0.001,-0.017,-0.008,-0.02,-0.017,-0.021,-0.061
Avg Ascent Slope (% per day),1.846,0.588,0.661,0.66,1.249,0.704,0.709,0.86,0.696,0.837,0.726,0.697,0.721,0.739,0.89,0.777,0.719,0.724,0.784,0.806,0.816,0.765,0.758,0.743,0.806,0.782,1.118,1.047,0.93,0.832,0.882
Ulcer Index (%),5.42,2.82,2.92,3.74,3.48,3.95,4.04,6.13,4.17,6.18,4.89,5.03,5.51,5.53,8.16,6.11,5.95,6.3,6.51,7.67,7.73,6.91,7.42,8.5,8.47,9.74,8.96,17.86,12.39,15.13,16.81
Martin Ratio,4.196,4.135,4.099,3.098,2.947,2.276,1.836,1.72,1.468,1.137,1.019,1.013,0.761,0.743,0.722,0.67,0.658,0.596,0.538,0.472,0.438,0.272,0.251,-0.001,-0.051,-0.179,-0.29,-0.349,-0.426,-0.436,-0.556


In [129]:
# 2016-2018 Bull Run
recovery_stats = analyze_recovery_time(my_tickers, start_date="2016-08-23", end_date="2018-07-19")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
display(recovery_stats.T)

$AVUV: possibly delisted; no price data found  (1d 2016-08-23 -> 2018-07-19) (Yahoo error = "Data doesn't exist for startDate = 1471924800, endDate = 1531972800")
$QGRO: possibly delisted; no price data found  (1d 2016-08-23 -> 2018-07-19) (Yahoo error = "Data doesn't exist for startDate = 1471924800, endDate = 1531972800")
$QQQM: possibly delisted; no price data found  (1d 2016-08-23 -> 2018-07-19) (Yahoo error = "Data doesn't exist for startDate = 1471924800, endDate = 1531972800")
$DYNF: possibly delisted; no price data found  (1d 2016-08-23 -> 2018-07-19) (Yahoo error = "Data doesn't exist for startDate = 1471924800, endDate = 1531972800")
$VFLO: possibly delisted; no price data found  (1d 2016-08-23 -> 2018-07-19) (Yahoo error = "Data doesn't exist for startDate = 1471924800, endDate = 1531972800")
$AVEM: possibly delisted; no price data found  (1d 2016-08-23 -> 2018-07-19) (Yahoo error = "Data doesn't exist for startDate = 1471924800, endDate = 1531972800")
$DUHP: possibly delist

,SPGP,QQQ,SPMO,VONG,IMCG,IJR,VOT,GSLC,VTI,IJH,MDY,VO,QUAL,COWZ,RSP,DIVO,VT,USMV,QVAL,VOE,CALF,MOAT,FNDX,VTV,VYM,IWD,SCHD,EMGF,VOOV,VXUS,IEMG,IMTM,EEMV,VDC,EMXC
Rank,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35
CAGR (%),26.31,26.31,22.66,22.02,19.42,20.58,15.51,16.27,16.78,15.35,15.04,13.91,15.13,16.6,13.44,14.56,13.9,11.34,18.07,12.42,17.73,15.48,13.35,13.83,11.74,10.81,12.67,14.0,11.48,11.01,12.02,8.3,7.69,1.09,-0.81
Avg Recovery (Trading Days),6.5,7.2,11.4,7.6,8.8,12.9,8.7,9.0,9.0,12.8,12.8,12.0,11.9,11.9,11.5,13.5,8.3,12.6,15.6,14.2,13.8,10.4,10.7,10.9,12.5,13.1,12.1,13.3,11.9,9.9,12.0,12.8,14.2,21.2,17.5
P95 Recovery (Trading Days),24.8,27.5,44.4,26.1,35.4,44.9,33.6,35.9,37.4,56.4,56.9,41.5,59.6,73.3,51.0,58.6,22.5,57.5,83.0,56.6,51.8,41.8,58.2,51.0,58.0,72.0,55.7,64.8,63.6,29.1,47.4,66.7,80.6,119.0,66.2
Ascent/Drawdown Area Ratio,0.094,0.084,0.078,0.079,0.072,0.053,0.059,0.049,0.052,0.047,0.045,0.042,0.042,0.04,0.038,0.036,0.045,0.032,0.03,0.035,0.036,0.034,0.034,0.034,0.03,0.027,0.029,0.035,0.027,0.033,0.028,0.021,0.021,0.008,0.017
Sortino Ratio,2.046,1.811,1.215,1.696,1.553,1.608,1.131,1.233,1.272,1.127,1.095,1.016,1.128,1.282,0.963,0.845,1.028,0.824,1.369,0.872,1.156,1.167,0.948,0.97,0.758,0.666,0.841,0.765,0.75,0.702,0.62,0.353,0.332,-0.434,-0.473
Return / Recovery Time,4.064,3.646,1.987,2.891,2.22,1.598,1.781,1.799,1.869,1.204,1.174,1.162,1.27,1.392,1.173,1.077,1.683,0.903,1.156,0.872,1.282,1.481,1.248,1.267,0.941,0.825,1.047,1.055,0.966,1.11,1.004,0.648,0.54,0.052,-0.046
Avg Ascent Slope (% per day),0.611,0.657,0.748,0.52,0.557,0.651,0.533,0.469,0.465,0.546,0.559,0.486,0.455,0.588,0.462,0.583,0.496,0.385,0.681,0.513,0.875,0.503,0.495,0.484,0.44,0.471,0.454,0.784,0.484,0.561,0.787,0.553,0.597,0.492,0.869
Ulcer Index (%),2.16,2.58,2.27,2.31,2.24,2.63,2.35,2.57,2.67,2.72,2.74,2.55,2.79,3.16,2.93,3.33,3.22,2.72,4.36,3.05,4.37,3.86,3.37,3.55,3.61,3.66,4.33,4.85,4.02,4.08,5.62,4.55,4.68,5.9,7.14
Martin Ratio,12.158,10.178,9.988,9.524,8.677,7.816,6.595,6.336,6.288,5.638,5.497,5.446,5.431,5.254,4.59,4.377,4.316,4.168,4.141,4.078,4.06,4.008,3.965,3.891,3.252,2.954,2.928,2.884,2.852,2.701,2.139,1.823,1.643,0.185,-0.114


In [130]:
# 2018 Q4 Correction
recovery_stats = analyze_recovery_time(my_tickers, start_date="2018-07-20", end_date="2019-08-19")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
display(recovery_stats.T)

$QQQM: possibly delisted; no price data found  (1d 2018-07-20 -> 2019-08-19) (Yahoo error = "Data doesn't exist for startDate = 1532059200, endDate = 1566187200")
$AVUV: possibly delisted; no price data found  (1d 2018-07-20 -> 2019-08-19) (Yahoo error = "Data doesn't exist for startDate = 1532059200, endDate = 1566187200")
$VFLO: possibly delisted; no price data found  (1d 2018-07-20 -> 2019-08-19) (Yahoo error = "Data doesn't exist for startDate = 1532059200, endDate = 1566187200")
$AVEM: possibly delisted; no price data found  (1d 2018-07-20 -> 2019-08-19) (Yahoo error = "Data doesn't exist for startDate = 1532059200, endDate = 1566187200")
$DUHP: possibly delisted; no price data found  (1d 2018-07-20 -> 2019-08-19) (Yahoo error = "Data doesn't exist for startDate = 1532059200, endDate = 1566187200")
$AVDV: possibly delisted; no price data found  (1d 2018-07-20 -> 2019-08-19) (Yahoo error = "Data doesn't exist for startDate = 1532059200, endDate = 1566187200")

6 Failed downloads:
[

,USMV,DYNF,VDC,DSTL,DIVO,MOAT,SCHD,IMCG,QUAL,VOT,VONG,VYM,GSLC,VOOV,VTV,SPMO,VTI,QQQ,IWD,QGRO,SPGP,VO,RSP,FNDX,VT,IMTM,VOE,EEMV,IJH,MDY,IJR,VXUS,COWZ,EMXC,IEMG,EMGF,CALF,QVAL
Rank,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
CAGR (%),16.35,8.47,13.63,12.18,8.04,8.26,5.71,8.12,6.1,6.35,6.43,3.73,4.38,3.75,3.39,4.43,3.48,4.21,2.54,3.27,2.59,2.24,1.78,1.71,-0.64,-1.63,-1.99,-2.01,-4.28,-4.38,-10.75,-5.86,-6.98,-5.46,-7.44,-11.08,-21.28,-15.85
Avg Recovery (Trading Days),7.9,7.5,10.9,11.2,12.2,9.2,14.7,13.7,15.9,15.9,18.8,13.9,18.7,15.8,18.7,25.5,18.7,24.7,20.2,20.9,14.3,18.9,16.3,24.6,25.5,52.6,23.3,29.0,52.2,65.0,31.9,134.0,52.2,52.8,66.2,134.0,52.6,66.2
P95 Recovery (Trading Days),14.7,24.0,26.0,46.8,35.5,42.2,79.5,45.4,67.9,62.4,74.4,54.8,78.6,69.3,80.6,108.7,77.2,103.1,85.8,89.2,50.6,78.2,68.6,113.4,94.1,160.6,105.5,99.2,195.2,206.9,158.1,253.7,184.8,155.2,163.0,253.7,199.0,209.0
Ascent/Drawdown Area Ratio,0.05,0.07,0.041,0.038,0.021,0.03,0.016,0.016,0.013,0.015,0.011,0.015,0.011,0.012,0.012,0.007,0.011,0.009,0.011,0.008,0.014,0.01,0.011,0.008,0.007,0.002,0.005,0.007,0.002,0.002,0.002,0.001,0.003,0.004,0.003,0.001,0.001,0.001
Sortino Ratio,1.2,0.368,0.941,0.582,0.31,0.29,0.081,0.233,0.101,0.113,0.107,-0.101,-0.034,-0.092,-0.126,-0.024,-0.104,-0.035,-0.209,-0.082,-0.138,-0.212,-0.258,-0.262,-0.491,-0.635,-0.617,-0.747,-0.746,-0.754,-1.158,-1.054,-0.93,-0.828,-0.957,-1.196,-1.973,-1.54
Return / Recovery Time,2.073,1.136,1.25,1.088,0.661,0.897,0.389,0.592,0.383,0.4,0.342,0.268,0.234,0.237,0.181,0.174,0.186,0.17,0.125,0.157,0.181,0.118,0.109,0.069,-0.025,-0.031,-0.086,-0.069,-0.082,-0.067,-0.337,-0.044,-0.134,-0.103,-0.112,-0.083,-0.405,-0.239
Avg Ascent Slope (% per day),0.506,0.576,0.629,0.679,0.565,0.615,0.654,0.775,0.722,0.821,0.78,0.578,0.696,0.639,0.611,0.784,0.681,0.906,0.612,0.841,0.838,0.697,0.615,0.663,0.6,0.583,0.619,0.575,0.729,0.733,0.82,0.622,0.811,0.89,0.88,0.888,0.98,0.839
Ulcer Index (%),2.98,2.0,3.57,4.54,3.86,4.28,5.21,7.56,6.24,6.69,6.88,4.44,5.84,5.38,4.92,7.47,5.97,7.48,5.14,7.78,7.11,6.4,5.81,6.01,5.82,7.9,6.67,3.94,8.02,8.03,12.77,6.91,7.85,5.51,7.3,10.23,16.14,11.76
Martin Ratio,5.481,4.226,3.813,2.681,2.084,1.93,1.097,1.074,0.978,0.949,0.935,0.842,0.75,0.696,0.688,0.592,0.584,0.562,0.494,0.421,0.364,0.35,0.306,0.284,-0.109,-0.206,-0.299,-0.51,-0.534,-0.546,-0.842,-0.848,-0.889,-0.99,-1.02,-1.083,-1.319,-1.348


In [131]:
# 2019 Rebound
recovery_stats = analyze_recovery_time(my_tickers, start_date="2019-08-20", end_date="2020-01-18")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
display(recovery_stats.T)

$QQQM: possibly delisted; no price data found  (1d 2019-08-20 -> 2020-01-18) (Yahoo error = "Data doesn't exist for startDate = 1566273600, endDate = 1579323600")
$VFLO: possibly delisted; no price data found  (1d 2019-08-20 -> 2020-01-18) (Yahoo error = "Data doesn't exist for startDate = 1566273600, endDate = 1579323600")
$DUHP: possibly delisted; no price data found  (1d 2019-08-20 -> 2020-01-18) (Yahoo error = "Data doesn't exist for startDate = 1566273600, endDate = 1579323600")

3 Failed downloads:
['QQQM', 'VFLO', 'DUHP']: possibly delisted; no price data found  (1d 2019-08-20 -> 2020-01-18) (Yahoo error = "Data doesn't exist for startDate = 1566273600, endDate = 1579323600")


,QUAL,QQQ,AVDV,GSLC,MOAT,VOOV,VT,DSTL,VONG,VXUS,VTI,VYM,FNDX,VTV,SPGP,DYNF,CALF,SCHD,IWD,EMGF,AVEM,DIVO,VOE,EMXC,RSP,IMTM,VO,IEMG,USMV,VOT,VDC,IJH,QVAL,MDY,SPMO,QGRO,COWZ,IJR,IMCG,AVUV,EEMV
Rank,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
CAGR (%),45.06,54.81,40.95,40.6,50.46,44.79,41.52,49.26,45.16,40.79,40.47,33.57,41.98,37.81,57.29,33.94,67.92,38.71,36.21,52.63,46.86,25.45,37.11,45.54,38.03,30.07,34.17,50.41,21.04,31.77,19.71,32.59,47.86,32.25,23.78,34.47,37.39,35.83,29.64,24.54,18.38
Avg Recovery (Trading Days),2.9,4.4,4.1,3.7,4.1,4.3,3.6,3.8,4.5,4.9,5.4,5.7,4.9,4.5,4.1,4.6,6.3,4.5,4.6,5.6,5.0,5.5,6.8,5.1,5.7,6.4,5.4,7.2,7.6,5.8,5.3,5.3,7.5,5.8,7.7,6.6,8.2,6.8,8.9,9.3,6.7
P95 Recovery (Trading Days),5.0,12.9,10.4,11.6,11.6,13.1,11.8,12.0,15.1,19.7,18.4,17.2,15.3,14.3,13.0,16.3,19.9,13.9,14.0,22.4,18.0,15.6,19.9,20.4,18.0,20.7,17.8,24.1,30.5,17.8,17.6,16.6,25.0,17.6,31.0,24.5,24.3,20.6,35.0,23.2,27.5
Ascent/Drawdown Area Ratio,0.443,0.319,0.363,0.376,0.345,0.326,0.332,0.323,0.294,0.283,0.279,0.288,0.278,0.279,0.286,0.292,0.201,0.265,0.259,0.189,0.198,0.259,0.23,0.211,0.232,0.184,0.209,0.164,0.144,0.17,0.173,0.186,0.136,0.181,0.124,0.125,0.121,0.141,0.098,0.104,0.088
Sortino Ratio,4.515,5.155,6.981,4.287,5.201,4.369,5.01,4.483,4.737,6.066,4.227,3.378,4.21,3.83,5.081,3.22,5.555,3.715,3.701,7.065,7.206,2.561,3.66,6.792,3.517,3.85,3.495,7.078,2.495,3.23,2.256,3.058,3.682,3.0,2.446,3.064,3.09,3.319,2.627,2.589,3.053
Return / Recovery Time,15.773,12.458,10.028,10.957,12.432,10.335,11.385,13.135,9.961,8.397,7.471,5.924,8.643,8.402,13.888,7.424,10.724,8.667,7.799,9.373,9.371,4.594,5.457,8.98,6.64,4.686,6.345,7.002,2.756,5.434,3.743,6.166,6.343,5.59,3.077,5.195,4.56,5.309,3.335,2.643,2.756
Avg Ascent Slope (% per day),0.538,0.737,0.464,0.547,0.69,0.479,0.472,0.621,0.643,0.474,0.552,0.41,0.521,0.426,0.792,0.46,0.781,0.489,0.426,0.671,0.553,0.419,0.485,0.653,0.498,0.459,0.521,0.602,0.405,0.604,0.403,0.587,0.848,0.585,0.483,0.647,0.617,0.686,0.676,0.659,0.444
Ulcer Index (%),0.93,1.24,0.93,0.92,1.17,1.05,0.98,1.16,1.09,1.04,1.08,0.9,1.14,1.06,1.61,0.98,1.97,1.16,1.11,1.63,1.52,0.83,1.22,1.52,1.27,1.05,1.22,1.82,0.86,1.31,0.83,1.47,2.18,1.47,1.24,1.9,2.1,2.15,1.87,1.82,1.67
Martin Ratio,48.561,44.285,44.106,43.952,42.982,42.515,42.424,42.326,41.532,39.175,37.416,37.133,36.76,35.766,35.64,34.802,34.436,33.335,32.658,32.231,30.781,30.766,30.374,29.911,29.901,28.603,28.052,27.776,24.357,24.215,23.706,22.171,21.915,21.881,19.131,18.096,17.848,16.675,15.843,13.516,11.015


In [132]:
# COVID-19 Pandemic
recovery_stats = analyze_recovery_time(my_tickers, start_date="2020-01-19", end_date="2021-09-21")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
display(recovery_stats.T)

$VFLO: possibly delisted; no price data found  (1d 2020-01-19 -> 2021-09-21) (Yahoo error = "Data doesn't exist for startDate = 1579410000, endDate = 1632196800")
$DUHP: possibly delisted; no price data found  (1d 2020-01-19 -> 2021-09-21) (Yahoo error = "Data doesn't exist for startDate = 1579410000, endDate = 1632196800")

2 Failed downloads:
['VFLO', 'DUHP']: possibly delisted; no price data found  (1d 2020-01-19 -> 2021-09-21) (Yahoo error = "Data doesn't exist for startDate = 1579410000, endDate = 1632196800")


,QQQM,QQQ,QGRO,VONG,IMCG,SPMO,VOT,DSTL,VTI,GSLC,QUAL,MOAT,COWZ,SCHD,CALF,DIVO,SPGP,DYNF,VO,VT,IMTM,RSP,VDC,FNDX,IJH,MDY,AVUV,IJR,VXUS,AVEM,USMV,EMGF,IEMG,AVDV,EMXC,VTV,IWD,VYM,VOE,VOOV,EEMV,QVAL
Rank,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42
CAGR (%),26.79,35.1,30.95,29.71,29.52,23.5,26.3,19.01,20.68,19.04,18.45,18.68,24.88,18.65,29.78,13.61,22.87,16.68,19.13,16.18,12.84,17.89,9.51,16.62,16.75,16.37,22.79,16.86,10.52,11.08,8.16,9.34,9.68,12.45,11.41,9.98,10.29,8.67,11.34,8.98,6.11,10.04
Avg Recovery (Trading Days),7.0,8.8,8.5,8.4,11.2,9.0,10.8,8.6,7.8,8.4,7.7,8.8,11.4,15.3,13.2,10.7,9.9,8.1,9.5,9.8,12.7,10.4,14.0,14.6,15.1,15.0,18.3,21.4,16.9,22.5,15.5,23.9,21.2,18.0,20.3,17.0,16.6,22.4,14.3,16.3,27.7,22.5
P95 Recovery (Trading Days),35.2,43.4,36.3,38.5,47.0,38.2,56.0,37.5,18.9,33.8,17.4,47.0,32.4,41.8,58.7,30.0,14.8,18.9,21.8,34.9,60.8,16.9,44.1,38.6,65.6,65.6,72.0,92.1,56.1,99.2,39.7,100.8,149.2,62.0,77.4,43.9,43.5,80.6,37.5,42.1,120.7,100.8
Ascent/Drawdown Area Ratio,0.081,0.044,0.039,0.039,0.031,0.032,0.031,0.032,0.028,0.027,0.029,0.028,0.022,0.025,0.025,0.028,0.021,0.021,0.022,0.023,0.023,0.018,0.019,0.015,0.015,0.015,0.013,0.012,0.013,0.012,0.01,0.011,0.013,0.009,0.01,0.01,0.009,0.008,0.01,0.008,0.008,0.006
Sortino Ratio,1.588,1.131,1.015,0.964,0.936,0.686,0.794,0.514,0.603,0.559,0.555,0.548,0.739,0.578,0.837,0.416,0.554,0.466,0.503,0.438,0.346,0.473,0.243,0.446,0.387,0.378,0.536,0.4,0.229,0.258,0.147,0.18,0.181,0.31,0.228,0.202,0.203,0.159,0.227,0.162,0.065,0.173
Return / Recovery Time,3.849,3.971,3.636,3.522,2.63,2.612,2.427,2.204,2.649,2.264,2.388,2.111,2.181,1.22,2.26,1.269,2.32,2.048,2.014,1.651,1.012,1.725,0.677,1.138,1.111,1.092,1.243,0.788,0.624,0.492,0.528,0.39,0.456,0.693,0.561,0.585,0.62,0.387,0.793,0.551,0.22,0.446
Avg Ascent Slope (% per day),0.878,1.374,1.396,1.287,1.367,1.406,1.327,1.113,1.131,1.168,1.116,1.125,1.221,1.163,1.754,0.955,1.355,1.191,1.163,1.052,1.025,1.165,1.014,1.177,1.371,1.364,1.781,1.551,1.011,1.095,0.993,1.135,1.202,1.022,1.15,1.183,1.217,1.185,1.267,1.095,0.796,1.28
Ulcer Index (%),3.04,6.83,7.39,7.16,8.03,6.8,8.27,7.27,8.33,7.79,7.58,7.76,10.47,8.09,12.96,6.11,11.08,8.5,9.85,8.48,7.12,10.19,5.62,10.39,11.81,11.81,16.62,14.2,9.57,10.55,8.28,9.54,10.04,13.49,12.37,10.83,11.57,10.3,13.69,11.09,7.75,17.12
Martin Ratio,8.808,5.137,4.188,4.148,3.676,3.459,3.18,2.614,2.482,2.445,2.434,2.406,2.375,2.306,2.298,2.23,2.063,1.961,1.942,1.908,1.804,1.755,1.693,1.599,1.419,1.386,1.372,1.187,1.1,1.05,0.986,0.98,0.964,0.923,0.922,0.921,0.89,0.842,0.828,0.81,0.788,0.586


In [133]:
# 2022 Rate Hike Drawdown
recovery_stats = analyze_recovery_time(my_tickers, start_date="2021-10-05", end_date="2024-02-26")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
display(recovery_stats.T)

,VFLO,DIVO,VOOV,COWZ,DUHP,DSTL,FNDX,VYM,VTV,QVAL,VDC,SPMO,MOAT,AVUV,DYNF,USMV,SPGP,QUAL,SCHD,IWD,GSLC,VTI,QQQM,QQQ,VONG,RSP,CALF,VOE,IJH,QGRO,MDY,VT,IMTM,AVDV,VO,VXUS,IMCG,IJR,EMXC,VOT,AVEM,EEMV,EMGF,IEMG
Rank,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44
CAGR (%),30.14,8.12,10.7,10.47,12.35,11.36,9.11,7.29,8.04,10.08,6.67,11.7,8.27,7.34,10.23,6.16,7.47,9.79,4.81,5.18,8.02,6.69,9.6,9.5,9.08,4.41,6.3,3.78,4.52,6.87,4.25,4.36,3.03,1.65,1.36,0.44,0.12,-0.33,-0.43,-1.11,-1.36,-1.3,-1.71,-4.37
Avg Recovery (Trading Days),8.1,24.5,19.4,22.9,21.5,22.6,23.7,25.3,22.0,30.2,43.8,29.1,35.4,44.5,37.1,37.3,37.9,30.8,33.4,35.4,30.8,37.5,43.2,40.1,37.3,56.9,38.5,57.4,72.2,47.8,64.4,63.6,58.4,97.5,144.8,84.0,116.0,84.1,98.5,96.8,148.5,84.0,147.8,148.2
P95 Recovery (Trading Days),34.0,140.0,87.9,109.8,84.0,75.5,80.6,149.6,89.4,93.5,223.0,77.5,168.8,195.2,181.8,162.1,184.6,95.4,113.6,144.5,95.0,165.8,214.2,187.5,166.7,296.0,176.4,276.0,364.7,248.0,337.6,334.8,316.0,409.5,484.2,404.6,456.0,403.7,408.2,427.8,501.1,382.1,459.0,500.9
Ascent/Drawdown Area Ratio,0.083,0.014,0.013,0.013,0.011,0.009,0.009,0.01,0.01,0.007,0.006,0.006,0.006,0.006,0.005,0.005,0.005,0.004,0.005,0.005,0.004,0.003,0.003,0.003,0.003,0.003,0.004,0.003,0.003,0.002,0.002,0.002,0.002,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001
Sortino Ratio,4.113,0.384,0.551,0.413,0.59,0.545,0.377,0.247,0.315,0.307,0.176,0.541,0.261,0.168,0.419,0.137,0.218,0.371,0.001,0.035,0.255,0.144,0.289,0.28,0.268,-0.033,0.092,-0.083,-0.02,0.13,-0.039,-0.037,-0.157,-0.27,-0.254,-0.407,-0.309,-0.372,-0.482,-0.362,-0.53,-0.804,-0.634,-0.8
Return / Recovery Time,3.713,0.332,0.553,0.457,0.575,0.503,0.384,0.288,0.366,0.334,0.152,0.403,0.234,0.165,0.276,0.165,0.197,0.318,0.144,0.146,0.26,0.178,0.222,0.237,0.243,0.077,0.164,0.066,0.063,0.144,0.066,0.069,0.052,0.017,0.009,0.005,0.001,-0.004,-0.004,-0.012,-0.009,-0.016,-0.012,-0.029
Avg Ascent Slope (% per day),0.679,0.635,0.798,0.951,0.888,0.882,0.854,0.741,0.755,1.269,0.676,0.871,1.058,1.244,0.941,0.69,0.963,1.02,0.736,0.822,0.922,0.925,1.259,1.25,1.18,0.895,1.304,0.848,1.05,1.154,1.068,0.859,0.865,0.869,0.983,0.844,1.137,1.131,0.852,1.206,0.933,0.598,0.917,0.92
Ulcer Index (%),2.59,4.57,6.26,6.52,7.9,7.78,6.74,5.4,5.99,9.98,6.65,11.68,9.18,8.55,13.25,8.13,9.95,13.43,6.82,7.65,12.37,12.82,18.87,18.91,18.16,9.67,14.52,8.8,11.63,17.94,11.77,12.62,15.34,11.34,15.69,13.66,22.08,15.64,16.23,24.12,15.78,12.22,15.7,18.99
Martin Ratio,11.646,1.778,1.711,1.607,1.564,1.459,1.35,1.349,1.343,1.01,1.004,1.002,0.901,0.859,0.772,0.757,0.751,0.73,0.706,0.677,0.648,0.522,0.509,0.502,0.5,0.456,0.434,0.43,0.388,0.383,0.361,0.346,0.198,0.145,0.087,0.032,0.005,-0.021,-0.026,-0.046,-0.086,-0.107,-0.109,-0.23


In [134]:
# 2024–2025 Bull Run
recovery_stats = analyze_recovery_time(my_tickers, start_date="2024-02-27", end_date="2025-01-18")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
display(recovery_stats.T)

,DYNF,SPMO,VTI,QGRO,GSLC,VYM,DIVO,VFLO,FNDX,VONG,VT,VTV,VO,QUAL,VOT,USMV,DUHP,IWD,VOE,QQQM,QQQ,IMCG,RSP,SCHD,IJH,MDY,VOOV,QVAL,MOAT,VDC,DSTL,SPGP,IJR,AVUV,COWZ,AVDV,EEMV,EMGF,IEMG,AVEM,VXUS,IMTM,EMXC,CALF
Rank,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44
CAGR (%),25.76,33.32,21.68,29.04,20.97,19.83,16.5,24.96,17.51,27.01,15.14,16.87,18.55,15.85,20.06,13.78,15.62,16.19,17.03,22.65,22.58,18.24,14.49,13.89,16.34,16.12,12.09,13.74,11.73,9.28,12.32,12.27,13.54,15.14,11.98,10.11,6.26,7.91,6.82,6.54,4.88,4.74,3.01,-6.53
Avg Recovery (Trading Days),8.3,11.6,7.2,10.4,7.4,7.5,8.4,11.9,8.2,10.1,7.9,7.1,9.4,7.3,12.7,8.8,8.0,8.2,9.5,13.6,13.6,10.7,8.7,8.1,15.5,18.3,9.6,12.6,10.6,12.5,9.6,15.6,16.0,17.4,14.1,9.3,16.2,14.1,16.5,14.1,13.4,14.6,15.2,55.0
P95 Recovery (Trading Days),37.0,42.0,31.0,31.8,31.5,32.0,28.0,43.8,31.7,40.5,27.0,32.0,34.0,28.2,52.8,33.0,28.6,31.0,36.1,52.4,52.4,33.9,32.0,30.5,41.4,58.5,34.2,36.2,41.9,42.4,34.1,62.4,44.0,52.0,52.1,25.0,57.7,57.0,59.0,57.0,46.2,59.4,59.6,173.6
Ascent/Drawdown Area Ratio,0.093,0.069,0.084,0.069,0.087,0.078,0.078,0.058,0.072,0.059,0.071,0.069,0.059,0.068,0.051,0.069,0.062,0.062,0.059,0.046,0.046,0.05,0.054,0.054,0.042,0.04,0.053,0.045,0.047,0.056,0.044,0.031,0.037,0.031,0.029,0.04,0.032,0.03,0.031,0.032,0.031,0.024,0.023,0.003
Sortino Ratio,1.94,2.14,1.667,2.118,1.637,1.966,1.82,2.325,1.547,1.608,1.129,1.666,1.537,1.171,1.393,1.379,1.324,1.51,1.556,1.287,1.278,1.313,1.205,1.272,1.097,1.078,1.035,0.96,0.816,0.782,1.087,0.718,0.716,0.779,0.795,0.513,0.237,0.302,0.202,0.159,0.009,-0.005,-0.173,-0.926
Return / Recovery Time,3.091,2.866,3.028,2.796,2.843,2.644,1.957,2.091,2.14,2.672,1.923,2.383,1.984,2.165,1.584,1.558,1.953,1.979,1.797,1.66,1.655,1.71,1.663,1.718,1.057,0.882,1.262,1.093,1.111,0.74,1.29,0.786,0.846,0.869,0.852,1.088,0.385,0.562,0.413,0.463,0.364,0.325,0.197,-0.119
Avg Ascent Slope (% per day),0.668,0.946,0.634,0.79,0.623,0.537,0.433,0.655,0.538,0.816,0.613,0.5,0.638,0.642,0.696,0.415,0.608,0.536,0.544,0.868,0.866,0.704,0.513,0.524,0.696,0.697,0.516,0.676,0.549,0.438,0.568,0.703,0.858,0.907,0.658,0.75,0.452,0.807,0.802,0.802,0.694,0.819,0.735,0.95
Ulcer Index (%),2.35,3.44,2.26,3.06,2.29,2.18,2.05,3.2,2.31,3.63,2.15,2.52,2.79,2.43,3.12,2.19,2.52,2.7,3.04,4.09,4.09,3.34,2.74,2.96,3.57,3.59,2.73,3.19,2.72,2.26,3.2,3.63,4.11,4.84,3.97,3.71,3.07,4.9,4.63,4.68,3.88,4.22,4.85,6.93
Martin Ratio,10.963,9.692,9.59,9.494,9.171,9.077,8.039,7.808,7.574,7.432,7.034,6.689,6.657,6.516,6.439,6.3,6.21,5.99,5.594,5.541,5.515,5.459,5.281,4.693,4.584,4.496,4.431,4.312,4.307,4.106,3.847,3.379,3.295,3.127,3.021,2.726,2.039,1.616,1.472,1.399,1.258,1.123,0.62,-0.941


In [135]:
# 2025 Trump Trade War
recovery_stats = analyze_recovery_time(my_tickers, start_date="2025-01-19", end_date="2025-07-26")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
display(recovery_stats.T)

,AVDV,IMTM,VXUS,AVEM,EMGF,IEMG,EEMV,EMXC,SPMO,VT,VDC,USMV,DIVO,DYNF,VOT,QQQM,QQQ,VO,DUHP,GSLC,VONG,VYM,VTI,RSP,IWD,VTV,VOOV,QGRO,IMCG,VOE,FNDX,MOAT,QUAL,SPGP,VFLO,DSTL,IJH,MDY,SCHD,COWZ,AVUV,IJR,QVAL,CALF
Rank,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44
CAGR (%),59.57,41.37,37.37,42.58,38.05,37.34,19.56,28.52,32.3,20.5,11.38,9.27,12.08,15.28,17.49,16.68,16.65,10.69,11.24,11.81,15.36,8.21,11.13,7.95,7.22,6.54,6.68,12.38,9.24,5.38,6.32,7.93,6.07,1.9,0.8,-0.48,-3.12,-3.24,-2.63,-3.68,-8.98,-9.99,-6.94,-16.57
Avg Recovery (Trading Days),4.6,6.1,5.2,4.6,4.5,6.1,6.7,11.2,6.6,7.6,23.8,9.7,14.2,16.0,16.0,15.6,15.6,12.7,14.4,27.5,18.8,14.5,18.8,16.7,16.7,14.6,17.0,29.5,16.7,30.2,14.5,29.8,24.2,41.7,31.0,62.5,128.0,128.0,31.2,31.2,128.0,128.0,128.0,128.0
P95 Recovery (Trading Days),11.6,17.2,14.0,17.5,16.9,21.1,22.5,35.8,23.6,29.2,84.6,39.7,61.4,64.4,66.1,63.7,63.7,57.0,61.3,76.0,79.2,62.0,70.0,65.9,66.3,62.7,67.5,92.8,70.6,94.5,61.6,92.1,85.8,110.8,83.3,117.8,128.0,128.0,88.7,103.1,128.0,128.0,128.0,128.0
Ascent/Drawdown Area Ratio,0.218,0.155,0.148,0.121,0.127,0.106,0.114,0.074,0.043,0.037,0.029,0.038,0.023,0.015,0.016,0.014,0.014,0.018,0.015,0.014,0.012,0.016,0.01,0.016,0.015,0.016,0.011,0.009,0.01,0.012,0.012,0.007,0.008,0.003,0.005,0.001,0.0,0.0,0.006,0.002,0.0,0.0,0.0,0.0
Sortino Ratio,2.963,2.163,2.065,2.245,2.096,2.043,1.394,1.587,1.192,0.915,0.617,0.342,0.534,0.558,0.608,0.535,0.524,0.343,0.386,0.377,0.48,0.221,0.326,0.189,0.149,0.115,0.116,0.347,0.23,0.036,0.09,0.175,0.076,-0.138,-0.221,-0.368,-0.418,-0.424,-0.478,-0.478,-0.636,-0.767,-0.639,-0.974
Return / Recovery Time,13.01,6.754,7.22,9.298,8.455,6.155,2.934,2.541,4.895,2.692,0.478,0.959,0.847,0.955,1.093,1.071,1.069,0.844,0.782,0.429,0.816,0.566,0.591,0.475,0.432,0.447,0.393,0.42,0.553,0.178,0.436,0.266,0.251,0.046,0.026,-0.008,-0.024,-0.025,-0.084,-0.118,-0.07,-0.078,-0.054,-0.129
Avg Ascent Slope (% per day),0.794,0.807,0.827,0.948,0.907,0.931,0.506,0.852,1.26,0.861,0.645,0.742,0.793,0.881,1.137,1.147,1.161,1.017,1.012,1.004,1.191,0.904,0.94,0.961,0.919,0.891,0.873,1.205,1.033,0.924,0.972,1.181,0.935,1.138,1.062,1.042,1.116,1.152,0.965,0.96,1.179,1.224,1.095,1.237
Ulcer Index (%),2.68,2.43,2.71,3.24,3.09,3.26,1.76,3.28,6.64,4.63,2.82,2.31,3.76,6.6,7.87,8.35,8.37,5.5,6.09,6.41,8.66,4.81,6.8,4.91,4.63,4.45,4.88,9.39,7.3,4.64,5.63,7.78,6.38,8.74,5.92,5.52,9.08,9.05,5.76,7.78,11.76,11.17,7.65,15.47
Martin Ratio,22.222,17.011,13.815,13.128,12.324,11.463,11.086,8.694,4.867,4.43,4.042,4.017,3.211,2.316,2.224,1.996,1.989,1.944,1.844,1.843,1.774,1.707,1.635,1.617,1.56,1.471,1.368,1.319,1.266,1.16,1.124,1.018,0.952,0.217,0.135,-0.087,-0.344,-0.358,-0.457,-0.473,-0.764,-0.894,-0.907,-1.071


In [136]:
# AI Boom
recovery_stats = analyze_recovery_time(my_tickers, start_date="2025-07-27", end_date=None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
display(recovery_stats.T)

,AVDV,EMXC,DIVO,FNDX,VXUS,DYNF,VTV,IMTM,VT,AVEM,VOOV,QVAL,IWD,VFLO,VYM,VTI,IEMG,GSLC,QUAL,COWZ,VOE,EMGF,CALF,IJR,MOAT,QQQ,QQQM,DUHP,DSTL,AVUV,VONG,EEMV,RSP,IJH,MDY,QGRO,SPMO,USMV,SPGP,SCHD,VO,VDC,IMCG,VOT
Rank,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44
CAGR (%),49.18,42.13,23.13,26.87,28.77,26.68,21.16,30.37,24.5,29.97,20.98,33.33,20.71,29.62,19.86,22.71,29.78,19.53,20.61,19.85,17.17,25.88,23.9,25.66,17.84,25.67,25.7,14.51,15.94,25.02,22.96,10.56,11.56,14.01,13.72,14.12,14.21,6.25,10.85,9.06,6.87,-3.57,-2.48,-6.03
Avg Recovery (Trading Days),4.7,7.1,4.4,5.3,5.0,7.3,6.0,9.2,7.4,7.3,5.3,6.6,6.5,5.1,7.1,6.7,7.6,6.1,7.6,8.8,9.1,7.5,8.3,9.2,11.0,8.4,8.4,11.2,6.2,10.3,7.8,9.7,13.4,10.2,11.6,13.0,11.2,15.8,9.3,19.2,10.4,32.3,20.2,13.9
P95 Recovery (Trading Days),11.6,25.5,10.0,13.3,12.4,19.5,12.0,22.2,21.0,23.5,10.3,18.6,11.8,12.0,14.4,19.1,24.0,18.2,19.5,17.1,13.6,24.0,29.5,25.0,24.2,29.1,29.1,30.9,15.8,35.2,25.0,31.4,25.2,36.8,41.5,35.9,33.7,43.2,31.1,58.2,24.2,79.7,39.0,38.0
Ascent/Drawdown Area Ratio,0.307,0.161,0.244,0.202,0.192,0.157,0.173,0.133,0.151,0.15,0.188,0.14,0.159,0.156,0.162,0.147,0.114,0.148,0.123,0.121,0.12,0.107,0.101,0.126,0.102,0.087,0.087,0.088,0.107,0.096,0.08,0.072,0.073,0.079,0.074,0.051,0.045,0.043,0.066,0.039,0.051,0.013,0.015,0.017
Sortino Ratio,5.952,3.97,3.789,3.221,3.301,2.441,2.797,3.937,2.421,2.453,2.438,3.74,2.42,2.618,2.297,2.003,2.578,1.749,2.338,2.029,1.878,2.192,1.772,1.977,1.778,1.661,1.659,1.524,1.603,1.737,1.549,0.945,0.974,0.987,0.957,0.972,0.809,0.293,0.699,0.727,0.266,-1.353,-0.823,-1.04
Return / Recovery Time,10.538,5.942,5.242,5.038,5.753,3.668,3.527,3.293,3.327,4.12,3.984,5.038,3.205,5.78,2.803,3.406,3.9,3.214,2.7,2.255,1.887,3.43,2.889,2.79,1.622,3.056,3.06,1.29,2.565,2.421,2.936,1.092,0.861,1.37,1.18,1.086,1.263,0.395,1.166,0.472,0.658,-0.11,-0.123,-0.435
Avg Ascent Slope (% per day),0.501,0.729,0.472,0.538,0.548,0.603,0.601,0.591,0.605,0.757,0.544,0.819,0.574,0.747,0.537,0.647,0.76,0.589,0.631,0.604,0.55,0.702,0.767,1.043,0.725,0.804,0.8,0.536,0.606,0.868,0.747,0.38,0.598,0.871,0.885,0.71,0.675,0.404,0.713,0.586,0.643,0.574,0.703,0.695
Ulcer Index (%),1.14,1.67,0.94,1.1,1.29,1.27,1.05,1.52,1.25,1.56,1.11,1.78,1.12,1.64,1.12,1.36,1.93,1.31,1.38,1.34,1.24,1.97,1.94,2.15,1.61,2.37,2.38,1.37,1.61,2.69,2.5,1.28,1.55,1.94,1.96,2.11,2.8,1.24,2.27,2.02,1.78,4.27,2.61,3.53
Martin Ratio,43.192,25.233,24.716,24.423,22.305,21.076,20.218,20.026,19.612,19.193,18.977,18.767,18.468,18.066,17.692,16.64,15.439,14.905,14.889,14.784,13.838,13.119,12.324,11.936,11.111,10.824,10.799,10.603,9.921,9.294,9.199,8.248,7.461,7.233,6.99,6.702,5.068,5.059,4.78,4.477,3.868,-0.836,-0.953,-1.708
